In [1]:
from pyspark import SparkConf,SparkContext
from operator import add
from tqdm import tqdm_notebook as tqdm
from fractions import *
from math import exp
from pyspark import StorageLevel
from tempfile import NamedTemporaryFile

In [2]:
StorageLevel.MEMORY_AND_DISK

StorageLevel(True, True, False, False, 1)

# Todo
[ ] 修正權重  
[ ] Personal Page rank  

In [3]:
conf = SparkConf().setMaster('local').setAppName('Test_APP')
conf.set("spark.executor.cores", "1")
# conf = SparkConf().setMaster("local[*]").setAppName('MatrixMultiplication')

conf.set('spark.executor.memory', '24G') \
    .set('spark.driver.memory', '30G') \
    .set('spark.driver.maxResultSize', '20G')
sc = SparkContext(conf=conf)

In [4]:
# file = 'dev.txt'
# file = 'HatePolitics_u2u_interactive_graph/part-00000'
file = 'u2u_interactive_graph/part-00000'
beta = 0.8
error_range = 10**-10

In [5]:
# 讀入黑名單
# with open('data/Bad_id.txt') as f:
with open('new_bad_id') as f:
    
    BAD_ID = f.readlines()
    BAD_ID = [i.strip() for i in BAD_ID]
    print(BAD_ID[:10])
    BAD_ID = set(BAD_ID)
    BAD_ID_Number =len(BAD_ID)


['CLV518', 'SUPERBOWL', 'nook777', 'bouly', 'qqee', 'tonyselina', 'mattlane', 'Aptantion', 'goodjeff', 'hotelss']


# 資料讀取跟前處理
將資料讀入，並轉成 (key,[value, this_node_connect_to_which_node]) 的格式

In [6]:
%%time
# 單純的資料讀取跟前處理
def line_to_stochastic_matrix(line):
    return_value = eval(line)
#     source, dest = list(map(int, line.split('\t')))
#     return_value.append([source, -1])
#     return_value.append([dest, -1])
#     return_value.append([source, dest])
    return return_value


    
stochastic_matrix = sc.textFile(file).map(line_to_stochastic_matrix)
stochastic_matrix = stochastic_matrix.groupByKey().mapValues(sum)
N = stochastic_matrix.flatMap(lambda x:x[0]).distinct().count()

stochastic_matrix = stochastic_matrix.map(lambda x:(x[0][0], (x[0][1], x[1]))).groupByKey() # (source,[(dist,weight)])

for i in stochastic_matrix.take(5):
    print(i[0], list(i[1]))

aswq17558 [('ccc723723', 1), ('AFIE', 1), ('bencert', 1), ('diefishfish', 1), ('FlynnZhang', 1), ('semih', 1), ('nicholas0406', 6), ('GoldenDream', 1), ('LABOYS', 6), ('PPTplayer', 1), ('frankieW', 1), ('analiya', 1), ('Julian9x9x9', 1), ('goodman5566', 3), ('AGIknight', 1), ('mean', 3), ('ReDmango', 1), ('Supergreen', 1), ('lianpig5566', 1), ('Tiara5566', 1), ('wilson0937', 1), ('SnakeO', 1), ('hello456', 1), ('losojo', 1), ('HappyDiD', 1), ('NCKUer', 1), ('douCai', 1), ('raugeon', 1), ('leolu2013', 1), ('IbakaBlock', 1), ('DCHC', 1), ('laverne1', 1), ('ivorysoap', 2), ('cowardlyman', 1), ('hernest01', 1), ('voohong', 1), ('NAGI', 2), ('yms12709', 1), ('abadcafe', 1), ('turbomons', 2), ('p1ayouro1e', 1), ('blueadam193', 1), ('omanorboyo', 3), ('seabax', 1), ('Gunbuster', 1), ('jxzhe', 1), ('ase14985', 1), ('nike3508', 1), ('deanh', 1), ('LLsolo', 1), ('ALOVET', 1), ('gordongj9ek', 1), ('Xbox960', 1), ('I8DGC', 1), ('GTOKevin', 1), ('ipmen', 1), ('a187', 1), ('CuLiZn5566', 1), ('Santan

In [7]:
%%time
# sum([])
# BAD_ID = stochastic_matrix.map(lambda x:x[0]).collect()[:100]
# BAD_ID = set(BAD_ID)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.63 µs


In [8]:
%%time
def turn_to_stochastic_matrix(item, N, ban_id):
    source, dests = item
    dests = [[i[0], exp(-1 * ( (i[1]**2) / (2 * beta))) ] for i in dests]
    goto = []
    total_weight = sum([i[1] for i in dests])
    for i in dests:
        dest, weight = i
        if total_weight == 0:
            weight = 0
        else:
            weight = weight/total_weight
        

        goto.append([dest, weight])
    
#     return key, [Fraction(1, N), goto]
    if source in ban_id:
        return (source, [1, goto])
    else:
        return (source, [0, goto])
    
stochastic_matrix = stochastic_matrix.map(lambda x: turn_to_stochastic_matrix(x, N, BAD_ID))
for i in stochastic_matrix.take(5):
    print(i[0], list(i[1]))

aswq17558 [0, [['ccc723723', 0.0015760169597110714], ['AFIE', 0.0015760169597110714], ['bencert', 0.0015760169597110714], ['diefishfish', 0.0015760169597110714], ['FlynnZhang', 0.0015760169597110714], ['semih', 0.0015760169597110714], ['nicholas0406', 4.981602779634575e-13], ['GoldenDream', 0.0015760169597110714], ['LABOYS', 4.981602779634575e-13], ['PPTplayer', 0.0015760169597110714], ['frankieW', 0.0015760169597110714], ['analiya', 0.0015760169597110714], ['Julian9x9x9', 0.0015760169597110714], ['goodman5566', 1.0619118744193014e-05], ['AGIknight', 0.0015760169597110714], ['mean', 1.0619118744193014e-05], ['ReDmango', 0.0015760169597110714], ['Supergreen', 0.0015760169597110714], ['lianpig5566', 0.0015760169597110714], ['Tiara5566', 0.0015760169597110714], ['wilson0937', 0.0015760169597110714], ['SnakeO', 0.0015760169597110714], ['hello456', 0.0015760169597110714], ['losojo', 0.0015760169597110714], ['HappyDiD', 0.0015760169597110714], ['NCKUer', 0.0015760169597110714], ['douCai', 0.

In [9]:
BAD_ID_Number

1371

In [10]:
%%time
tmp2 = stochastic_matrix.filter(lambda x:x[0] in BAD_ID).count()
tmp2

CPU times: user 16 ms, sys: 4 ms, total: 20 ms
Wall time: 34 s


1344

In [11]:
# %%time
# tmp = stochastic_matrix.map(lambda x:x[1][0]).sum()
# tmp

In [12]:
# tmp2 = stochastic_matrix.map(lambda x:x[1]).collect()

In [13]:
# len(tmp2)


In [14]:
# tmp2 = set(tmp2)
# len(tmp2)

In [15]:
# BAD_ID_Number

In [16]:
# BAD_ID & tmp2

In [17]:
# %%time
# stochastic_matrix = stochastic_matrix.map(lambda x: turn_to_stochastic_matrix(x, N, BAD_ID))
# # [(i, list(j)) for i,j in stochastic_matrix.collect()]
# stochastic_matrix.take(5)

In [18]:
# 格式是
# source [(dist,(weight, vale))]
# for i in stochastic_matrix.take(5):
#     print(i[0], list(i[1]))

# Mapper
把所有 node 對其他 node 的貢獻都算出來

In [19]:
%%time
def get_new_value(item, beta, N, bad_id):
    source, v = item
    value, dests = v
#     weight, value = value
#     source, dest = key
#     total_weight = len([i[1][0] for i in dests])
    
    new_values = []
    for i in dests:
        dest, weight = i
#         new_values.append([i, beta * (value/ N_dest) + (1-beta)/N])
        # 跳到 bad_id (personal page rank)
#         if dest in bad_id: #
#             new_values.append([i[0], beta * (value * weight) +  ((1-beta)/len(bad_id))])
#         else:
#             new_values.append([i[0], beta * (value * weight)] )
        new_values.append([i[0], beta * (value * weight)] )
        
    return new_values

new_values = stochastic_matrix.flatMap(lambda x: get_new_value(x, beta, N, BAD_ID))
new_values.take(30)

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 918 ms


[['ccc723723', 0.0],
 ['AFIE', 0.0],
 ['bencert', 0.0],
 ['diefishfish', 0.0],
 ['FlynnZhang', 0.0],
 ['semih', 0.0],
 ['nicholas0406', 0.0],
 ['GoldenDream', 0.0],
 ['LABOYS', 0.0],
 ['PPTplayer', 0.0],
 ['frankieW', 0.0],
 ['analiya', 0.0],
 ['Julian9x9x9', 0.0],
 ['goodman5566', 0.0],
 ['AGIknight', 0.0],
 ['mean', 0.0],
 ['ReDmango', 0.0],
 ['Supergreen', 0.0],
 ['lianpig5566', 0.0],
 ['Tiara5566', 0.0],
 ['wilson0937', 0.0],
 ['SnakeO', 0.0],
 ['hello456', 0.0],
 ['losojo', 0.0],
 ['HappyDiD', 0.0],
 ['NCKUer', 0.0],
 ['douCai', 0.0],
 ['raugeon', 0.0],
 ['leolu2013', 0.0],
 ['IbakaBlock', 0.0]]

# Reducer
加總所有貢獻

In [20]:
%%time
def topic_rank(x, bad_id):
    if x[0] in bad_id:
        return [x[0], x[1] + ((1-beta)/len(bad_id)) + ((1-beta) * 1)]
    else:
        return [x[0], x[1]]
new_values = new_values.reduceByKey(add).map(lambda x: topic_rank(x, BAD_ID) )
new_values.take(5)

CPU times: user 28 ms, sys: 4 ms, total: 32 ms
Wall time: 52.6 s


[['ccc723723', 0.06739337981862062],
 ['FlynnZhang', 0.008854876843076556],
 ['LABOYS', 0.011995682951920402],
 ['PPTplayer', 1.0303927796063577],
 ['analiya', 0.009517801046225082]]

# 算出 S

In [21]:
# S = new_values.values().reduce(add)
# S

In [22]:
%%time
S = new_values.values().sum()
S

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 799 ms


1154.2576221735933

# 結合 new 跟 old 為之後的更新做準備

In [23]:
# def reducer(x):

new_stochastic_matrix = stochastic_matrix.fullOuterJoin(new_values)
new_stochastic_matrix.take(1)

# [(x, list(map(list, y))) for x, y in sorted(list(new_stochastic_matrix.collect()))]


[('aswq17558',
  ([0,
    [['ccc723723', 0.0015760169597110714],
     ['AFIE', 0.0015760169597110714],
     ['bencert', 0.0015760169597110714],
     ['diefishfish', 0.0015760169597110714],
     ['FlynnZhang', 0.0015760169597110714],
     ['semih', 0.0015760169597110714],
     ['nicholas0406', 4.981602779634575e-13],
     ['GoldenDream', 0.0015760169597110714],
     ['LABOYS', 4.981602779634575e-13],
     ['PPTplayer', 0.0015760169597110714],
     ['frankieW', 0.0015760169597110714],
     ['analiya', 0.0015760169597110714],
     ['Julian9x9x9', 0.0015760169597110714],
     ['goodman5566', 1.0619118744193014e-05],
     ['AGIknight', 0.0015760169597110714],
     ['mean', 1.0619118744193014e-05],
     ['ReDmango', 0.0015760169597110714],
     ['Supergreen', 0.0015760169597110714],
     ['lianpig5566', 0.0015760169597110714],
     ['Tiara5566', 0.0015760169597110714],
     ['wilson0937', 0.0015760169597110714],
     ['SnakeO', 0.0015760169597110714],
     ['hello456', 0.0015760169597110714]

In [24]:
new_stochastic_matrix.take(1)[0][1][1]

0.0

In [25]:
N

288408

# 更新舊的數值

In [26]:
def sum_new_value(item, S, N, estimate_S):
    key, value = item
    
    if value[1] == None :
        new_value = 0 + (estimate_S-S)/N
    else:
        new_value = value[1] + (estimate_S-S)/N

    
    if value[0] == None:
        return [key, [new_value, []]]
    else:
        return [key, [new_value, value[0][1]]]

estimate_S = len(BAD_ID) * (1 + 1 - beta)
new_stochastic_matrix = new_stochastic_matrix.map(lambda x: sum_new_value(x, S, N, estimate_S))

# sorted(stochastic_matrix.collect())[:10]
new_stochastic_matrix.take(10)

[['aswq17558',
  [0.0017022495139746704,
   [['ccc723723', 0.0015760169597110714],
    ['AFIE', 0.0015760169597110714],
    ['bencert', 0.0015760169597110714],
    ['diefishfish', 0.0015760169597110714],
    ['FlynnZhang', 0.0015760169597110714],
    ['semih', 0.0015760169597110714],
    ['nicholas0406', 4.981602779634575e-13],
    ['GoldenDream', 0.0015760169597110714],
    ['LABOYS', 4.981602779634575e-13],
    ['PPTplayer', 0.0015760169597110714],
    ['frankieW', 0.0015760169597110714],
    ['analiya', 0.0015760169597110714],
    ['Julian9x9x9', 0.0015760169597110714],
    ['goodman5566', 1.0619118744193014e-05],
    ['AGIknight', 0.0015760169597110714],
    ['mean', 1.0619118744193014e-05],
    ['ReDmango', 0.0015760169597110714],
    ['Supergreen', 0.0015760169597110714],
    ['lianpig5566', 0.0015760169597110714],
    ['Tiara5566', 0.0015760169597110714],
    ['wilson0937', 0.0015760169597110714],
    ['SnakeO', 0.0015760169597110714],
    ['hello456', 0.0015760169597110714],
  

In [27]:
S = new_stochastic_matrix.values().map(lambda x:x[0]).reduce(add)
S

1645.200000000156

In [28]:
%%time
def check_end(x, error_range):

    old, new = x[1]
    if old == None or new == None:
        return False
    return abs(old - new) < error_range

V_new_stochastic_matrix = new_stochastic_matrix.map(lambda x: (x[0],x[1][0]))
V_stochastic_matrix = stochastic_matrix.map(lambda x:(x[0],x[1][0]))
is_end = V_stochastic_matrix.fullOuterJoin(V_new_stochastic_matrix).map(lambda x:check_end(x, error_range)).sum() == V_new_stochastic_matrix.count()
# new_stochastic_matrix
# stochastic_matrix

CPU times: user 48 ms, sys: 8 ms, total: 56 ms
Wall time: 1min 11s


In [29]:
is_end


False

In [30]:
stochastic_matrix = new_stochastic_matrix

In [31]:
# stochastic_matrix.saveAsSequenceFile('tttttt')

In [32]:
# %%time
# tmpFile = NamedTemporaryFile(delete=True)
# tmpFile.close()
# stochastic_matrix = new_stochastic_matrix
# stochastic_matrix.saveAsTextFile(tmpFile.name)
# stochastic_matrix = sc.wholeTextFiles(tmpFile.name)

In [33]:
# new_stochastic_matrix.getNumPartitions()

In [34]:
# t = stochastic_matrix.map(lambda x:x[1])

In [35]:
# def fileToRDD(x):
#     x.strip()
#     x = x.split('\n')
#     x = list(filter(lambda x: x.strip() != '', x))
#     x = list(map(eval,x))
#     return x
# stochastic_matrix = stochastic_matrix.repartition(1).values().flatMap(fileToRDD)
# t = stochastic_matrix.repartition(1).values().flatMap(fileToRDD)

In [36]:
# t.getNumPartitions()

In [37]:
# t.take(1)

In [38]:
# t = sc.parallelize(t)

In [39]:
# new_stochastic_matrix.take(1)

In [40]:
# t.take(1)

In [41]:
# %%time
# stochastic_matrix = new_stochastic_matrix
# stochastic_matrix = stochastic_matrix.collect()
# stochastic_matrix = sc.parallelize(stochastic_matrix)

In [42]:
# %%time
# new_values = stochastic_matrix.flatMap(lambda x: get_new_value(x, beta, N, BAD_ID))

In [43]:
# %%time
# new_values = new_values.reduceByKey(add).map(lambda x: topic_rank(x, BAD_ID) )

In [44]:
# %%time
# S = new_values.values().sum()

# 上面那些流程多跑幾次，直到收斂

In [45]:
%%time
from time import time

# for idx, i in tqdm(enumerate(range(19))):
idx = 0
while not is_end:
    
    start_time = time() # <-- start timing
    
    new_values = stochastic_matrix.flatMap(lambda x: get_new_value(x, beta, N, BAD_ID))
#     print(f'{idx} iter')
#     new_values.cache()
    new_values = new_values.reduceByKey(add).map(lambda x: topic_rank(x, BAD_ID) )
    
#     print(f'{idx} check problem {new_values.count()}')
    S = new_values.values().sum()
    
#     print(f'{idx} check problem')
    new_stochastic_matrix = stochastic_matrix.fullOuterJoin(new_values)
    
#     S = new_values.values().sum()
#     print(f'{idx} check problem')
#     new_stochastic_matrix = new_stochastic_matrix.map(lambda x: sum_new_value(x, S, N))
    estimate_S + len(BAD_ID) * ( 1 - beta)
    new_stochastic_matrix = new_stochastic_matrix.map(lambda x: sum_new_value(x, S, N, estimate_S))

    
#     print(f'{idx} check problem')
    V_new_stochastic_matrix = new_stochastic_matrix.map(lambda x: (x[0],x[1][0]))
    
#     print(f'{idx} check problem')
    V_stochastic_matrix = stochastic_matrix.map(lambda x:(x[0],x[1][0]))
    
#     print(f'{idx} check problem')
    N_True = V_stochastic_matrix.fullOuterJoin(V_new_stochastic_matrix).map(lambda x:check_end(x, error_range)).sum()
    N_new = V_new_stochastic_matrix.count()
    is_end = (N_True == N_new)
#     print(f'{idx} iter end')
    
#     tmpFile = NamedTemporaryFile(delete=True)
#     tmpFile.close()
    stochastic_matrix = new_stochastic_matrix
    stochastic_matrix = stochastic_matrix.repartition(1) 
#     stochastic_matrix.saveAsTextFile(tmpFile.name)
#     stochastic_matrix = sc.wholeTextFiles(tmpFile.name)
#     stochastic_matrix = stochastic_matrix.repartition(1).values().flatMap(fileToRDD)

#     stochastic_matrix = stochastic_matrix.cache()
#     stochastic_matrix = stochastic_matrix.collect()
#     stochastic_matrix = sc.parallelize(stochastic_matrix)

        
    end_time = time()
    print(idx, end_time - start_time, N_True, N_new, new_values.count(), S)
    idx += 1

    

0 119.89038634300232 245944 288408 45281 950.0337702130653
1 198.25539445877075 0 288408 45281 1352.5043095855017
2 232.08179926872253 0 288408 45281 1339.3665640236145
3 232.9739100933075 0 288408 45281 1341.8203318609633
4 232.3912854194641 1 288408 45281 1342.1772847604261
5 233.43960642814636 1 288408 45281 1342.2958226542012
6 231.99774646759033 2 288408 45281 1342.3999789444936
7 231.4497606754303 1 288408 45281 1342.44027085435
8 231.7622730731964 0 288408 45281 1342.4640536547513
9 231.849769115448 0 288408 45281 1342.47918288359
10 231.98145055770874 1 288408 45281 1342.4902968802376
11 231.70974469184875 0 288408 45281 1342.4986836423334
12 232.26680755615234 0 288408 45281 1342.505304406602
13 232.4176652431488 0 288408 45281 1342.5105456055314
14 233.39968061447144 0 288408 45281 1342.5147484105717
15 232.8703420162201 0 288408 45281 1342.5181043965074
16 232.0658564567566 0 288408 45281 1342.5207972064682
17 232.21183586120605 0 288408 45281 1342.522950117495
18 231.993138

In [46]:
stochastic_matrix.saveAsTextFile('real_result_v2_test')

# 輸出結果

In [47]:
output = sorted(stochastic_matrix.collect(), key=lambda x: x[1][0], reverse=True)

with open('output_v2_test.txt', 'w') as f:
    for i in output:
        f.write(f'{i[0]}\t{i[1][0]}\n')

In [48]:
with open('output_v2_test.txt') as f:
    o = f.readlines()
    
for i in range(10):
    print(o[i].strip())

seabox	4.002685334828794
Bignana	3.6920587398675493
XXXXGAY	3.689452808974696
iPad3	3.431901242246264
Kay731	3.1341120276561867
mayaman	2.5881133446814735
okcool	2.031229532267024
hateOnas	1.988124459533813
gn01765288	1.8404985936426927
ReDmango	1.8225578085433631


seabox	4.002685334828794
zongyi	0.0010494454431404204
sunhouse	0.0010494454431404204
Ash1taka	0.0010494454431404204
JumjumpTiger	0.0010494454431404204
uma1121	0.0010494454431404204
nieg	0.0010494454431404204
Cain13	0.0010494454431404204
jiajiaWeng	0.0010494454431404204
yenzubob	0.0010494454431404204


In [49]:
# !pip install networkx
# stochastic_matrix.count()

In [50]:
# import networkx as nx
# G=nx.DiGraph()


In [51]:
# with open(file) as f:
#     edge = f.readlines()
#     edge = [i.strip().split('\t') for i in edge]
    
# G.add_edges_from(edge)
    

In [52]:
# for i in edge:
#     G.add_edge(*i)

In [53]:
# pr = nx.pagerank(G, alpha=0.8)
# pr = sorted(list(pr.items()), key=lambda x:x[1], reverse=True)

In [54]:
# pr[:20]

In [55]:

# from fractions import *
# a = Fraction(10,8)
# a/2

In [56]:
!head -n 10 output_v2_test.txt

seabox	4.002685334828794
Bignana	3.6920587398675493
XXXXGAY	3.689452808974696
iPad3	3.431901242246264
Kay731	3.1341120276561867
mayaman	2.5881133446814735
okcool	2.031229532267024
hateOnas	1.988124459533813
gn01765288	1.8404985936426927
ReDmango	1.8225578085433631


In [58]:
!tail -n 10 output_v2_test.txt

kitty1117	0.0010494454431404204
yenzubob	0.0010494454431404204
jiajiaWeng	0.0010494454431404204
Cain13	0.0010494454431404204
nieg	0.0010494454431404204
uma1121	0.0010494454431404204
JumjumpTiger	0.0010494454431404204
Ash1taka	0.0010494454431404204
sunhouse	0.0010494454431404204
zongyi	0.0010494454431404204
